In [6]:
#import libraries

import numpy as np # linear algebra
import pandas as pd # data processing

ds = pd.read_csv(r'/Volumes/sharedfolder/ftpdata/ds/data_full_cleaned.csv')

print('Data Read ok')

Data Read ok


In [12]:
#initial model to test ideas
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

X = ds[['Pressure[Bar]','DP[Bar]','Temperature[C]','Velocity[m/s]','LiqDen[kg/m3]','WaterCut[%]','choke','QgStd[m3/d]']].astype(float)
y = X.pop('QgStd[m3/d]')

# Train and score baseline model
model = RandomForestRegressor(criterion="mae", random_state=0)
score = cross_val_score(
    model, X, y, cv=5, scoring="neg_mean_absolute_error"
)
sc = score
score = -1 * score.mean()

print(sc)
print(f"MAE Score: {score:.4}")

[-4027.37759617 -1921.32364265 -1276.4303387  -1547.64500794
 -1668.98193392]
MAE Score: 2.088e+03


In [13]:
X = ds[['Pressure[Bar]','DP[Bar]','Temperature[C]','Velocity[m/s]','LiqDen[kg/m3]','WaterCut[%]','choke','QgStd[m3/d]']].astype(float)

y = X.pop('QgStd[m3/d]')



In [14]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)

In [1]:
#try with deep learning model

from keras.layers import Input, Dense, Concatenate
from keras.models import Sequential
from keras.models import Model

model = Sequential()
model.add(Dense(64, input_dim=len(X.columns), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_absolute_error')
model.summary()




ModuleNotFoundError: No module named 'keras'

In [ ]:
from keras.callbacks import Callback

# Display training progress by printing a single dot for each completed epoch
class PrintDot(Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')
    
#Visualize the model's training progress using the stats stored in the history object.
from keras.callbacks import History 
history = History()
    
    
history = model.fit(X_train,y_train, epochs=2000, batch_size=12, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
#Visualize the model's training progress using the stats stored in the history object.

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 80000])
  plt.xlabel('Epoch')
  plt.ylabel('Error [Gas]')
  plt.legend()
  plt.grid(True)

plot_loss(history)

In [ ]:
model.evaluate(X_valid,y_valid)

In [ ]:
print("Saving model")
# Save your model
import joblib
joblib.dump(model, 'gas_model.pkl')
print("Model dumped!")

In [3]:
# Load the model that you just saved
import joblib
model = joblib.load('gas_model.pkl')

ModuleNotFoundError: No module named 'keras'

In [28]:
test = pd.DataFrame([[42,0.57,27,31,714,11,14]])
#test = [[42,0.57,27,31,714,11,14]]

#test = pd.DataFrame([[42,0.57,27,31,714,11,14]])

#test = pd.DataFrame([{"Pressure[Bar]": 42, "DP[Bar]": 0.57, "Temperature[C]": 27, "Velocity[m/s]":31, "LiqDen[kg/m3]":712, "WaterCut[%]":12, "choke": 14]})


data = "42,0.57,27,31,714,11,16"

df = pd.DataFrame([x.split(',') for x in data.split('\n')])
print(df)

#test
#print(test.shape)
print("Gas flow rate predicted:", model.predict(df), "Sm3/d")

    0     1   2   3    4   5   6
0  42  0.57  27  31  714  17  16
Gas flow rate predicted: [84828.48534457] Sm3/d
